In this notebook, we'll convert some EPhys data from openephys(legacy) to the Neurodata Without Borders (.nwb) format.

Use a virtual enviroment with the following dependencies:
- neuroconv

Each experimental input is associated to an `Interface`. So each of your data files will be associated with an `Interface`. The metadata associated to each input depends on its `Interface' type, so you should make sure that you're using the right one.

We'll first make a .nwb file containing only one input. This should clarify the basic idea. Then we'll combine a few inputs.

# One input

First, we need to import some useful tools from the `neuroconv` library. These deal with the structure of the metadata

In [1]:
from neuroconv.utils.dict import load_dict_from_file, dict_deep_update

We'll now import the recording. This will be associated with an `Interface`. The correct `Interface` for a recording in the OpenEphys format is a `OpenEphysRecordingInterface`.

In [2]:
from neuroconv.datainterfaces import OpenEphysRecordingInterface

We point this `Interface` at our recording, by directing it to the directory that the recording is in.

In [3]:
from pathlib import Path

folder_path = f"somedata/stream/"
data_name = ''

# Note: if there are multiple streams, like CH and ADC, you need to tell it which stream to use
my_ephys_interface = OpenEphysRecordingInterface(folder_path=folder_path, stream_name="Signals CH")

We've now created an `Interface` called `my_ephys_interface`. This has already taken a look at our data, made sure it can read it, and extracted any metadata it can. We can take a look at this metadata. It is stored in a `DeepDict` which is a custom `Dict`.

In [4]:
existing_data = my_ephys_interface.get_metadata()

/Users/chris/.my_venvs/SpikeDev/lib/python3.11/site-packages/neuroconv/datainterfaces/ecephys/openephys/openephyslegacydatainterface.py:76: UserWarning: The timestamp for starting time from openephys metadata is ambiguous ('12:38:29')! Only the date will be auto-populated in metadata. Please update the timestamp manually to record this value with the highest known temporal resolution.
  warn(


In my experience, it doesn't do a very good job of extracting metadata. There is a LOT more metadata we can input. All the options are stored in a **schema**: a schema tells you the structure of a database (or dictionary). We can check out the schema, though it is a bit intimidating to look at:

In [5]:
my_ephys_interface.get_metadata_schema()

Gross. So that you don't have to use this, I've made a yaml from schema function. This takes in your interface and the path to where you want to save the metadata. There are two optional arguments: `descriptions = True` or `False` and `existing_data` which will input any existing data into the yaml file.

In [6]:
def make_yaml_from_schema(interface, metadata_path, descriptions=True,existing_data=None):
    
    the_schema = interface.get_metadata_schema()
    
    metadata_file = open(metadata_path, 'w')
    
    for level1key in list(the_schema['properties'].keys()):
    
        print(f"{level1key}:", file=metadata_file)
    
        level2keys = list(the_schema['properties'][level1key]['properties'].keys())
        if 'definitions' in level2keys:
            level2keys.remove('definitions')
            
        for level2key in level2keys:
    
            print( f"  {level2key}: ", end="", file=metadata_file)
    
            if existing_data is not None and level2key in existing_data[level1key] and type(existing_data[level1key][level2key]) == str:            
                print(f"{existing_data[level1key][level2key]}" , end="", file=metadata_file)
                
            if descriptions == True:
                try:
                    print(f" # {the_schema['properties'][level1key]['properties'][level2key]['description']}" , file=metadata_file, end="")
                except:
                    if level1key != 'Ecephys':
                        print(f" # No description given",  file=metadata_file, end="")
            
            print("", file=metadata_file)
            
            if the_schema['properties'][level1key]['properties'][level2key]['type'] == 'array':
    
                if '$ref' in the_schema['properties'][level1key]['properties'][level2key]['items']:
                    print(f"    - {{\n", end="", file=metadata_file)
                    for level3key in the_schema['properties']['Ecephys']['properties']['definitions'][level2key]['properties'].keys():
                        
                        print(f"    {level3key}: ,", file=metadata_file, end="")
                        #print(f"{level1key}, {level2key}, {level3key}", file=metadata_file, end="")
                                
                        if descriptions == True:
                            try:
                                print(f" # {the_schema['properties'][level1key]['properties']['definitions'][level2key]['properties'][level3key]['description']}", file=metadata_file, end="")
                            except:
                                print(f" # No description given", file=metadata_file, end="")
                        print("", file=metadata_file)
                    
                    print(f"}}", file=metadata_file)
                
                else:
                    print(f"    -", file=metadata_file)
    
            if level2key == 'ElectricalSeries':
    
                #print("", file=metadata_file)
               
                for level3key in list(the_schema['properties'][level1key]['properties'][level2key]['properties'].keys()):
                    print( f"    {level3key}: ", end="", file=metadata_file)
    
                    if existing_data is not None and level3key in existing_data[level1key][level2key] and type(existing_data[level1key][level2key][level3key]) == str:            
                        print(f"{existing_data[level1key][level2key][level3key]}" ,end="", file=metadata_file)
                
                    
                    if descriptions == True:
                        try:
                            print(f" # {the_schema['properties'][level1key]['properties'][level2key]['properties'][level3key]['description']}", file=metadata_file, end="")
                        except:
                            print(f" # No description given", file=metadata_file, end="")
                    
                    print("", file=metadata_file)
                
    metadata_file.close()

Below are a few examples. Go and explore the YAML files! Any simple text edit can open them.

Note: you can also update the metadata directly in Python by editing the `metadata` dictionary. For more details, see the Neuroconv documentation (https://neuroconv.readthedocs.io/en/main/user_guide/nwbconverter.html).

In [7]:
make_yaml_from_schema(my_ephys_interface, "metadata_plain.yml", descriptions=False)
make_yaml_from_schema(my_ephys_interface, "metadata_descriptions.yml", descriptions=True)
make_yaml_from_schema(my_ephys_interface, "metadata_descriptions_withdata.yml", descriptions=True, existing_data = existing_data)

Our data folder is set. Our metadata has been filled out. We're ready to make a .nwb file! Note that some of the metadata is __required__: the .nwb file will not compile if you've not supplied it. If this error comes up, just go and update the .yml file.

We use the `run_conversion` function to run the file conversion.

In [9]:
nwbfile_path = "nwbdata/harry.nwb"  # Where to save the .nwb file
metadata_path = "metadata_updated.yml"

metadata_from_yaml = load_dict_from_file(file_path=metadata_path)

if isinstance(metadata_from_yaml, dict):
    my_ephys_interface.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata_from_yaml)

NWB file saved at nwbdata/harry.nwb!


# Several inputs

In [62]:
nwbfile_path = "nwbdata/harry.nwb"  # This should be something like: "./saved_file.nwb"
interface.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata)

NWB file saved at nwbdata/harry.nwb!


In [51]:
metadata['NWBFile']

DeepDict: {'session_description': 'Auto-generated by neuroconv', 'identifier': 'ddbf1f5b-3ff0-4d02-b1ab-0f3cde33d553', 'session_start_time': datetime.datetime(2023, 10, 30, 12, 38, 29, tzinfo=tzfile('/usr/share/zoneinfo/GMT'))}

In [57]:
from datetime import datetime
from dateutil import tz
